# Proyecto 2 - Visión por Computadora
## Ejercicio 2
## Integrantes:

- Javier Alvarado - 21188
- Mario Guerra - 21008
- Emilio Solano - 21212

In [1]:
!pip install opencv-python mediapipe torch torchvision
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt

  Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached mediapipe-0.10.21-cp312-cp312-win_amd64.whl.metadata (10 kB)
  Using cached torch-2.6.0-cp312-cp312-win_amd64.whl.metadata (28 kB)
  Using cached torchvision-0.21.0-cp312-cp312-win_amd64.whl.metadata (6.3 kB)
  Using cached numpy-2.2.4-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached absl_py-2.2.2-py3-none-any.whl.metadata (2.6 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached jax-0.5.3-py3-none-any.whl.metadata (22 kB)
  Using cached jaxlib-0.5.3-cp312-cp312-win_amd64.whl.metadata (1.2 kB)
  Using cached matplotlib-3.10.1-cp312-cp312-win_amd64.whl.metadata (11 kB)
  Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata (61 kB)
  Using cached opencv_contrib_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached protobuf-4.25.6-cp310-abi3-win_amd64.whl.m

fatal: destination path 'yolov5' already exists and is not an empty directory.


  Using cached PyYAML-6.0.2-cp312-cp312-win_amd64.whl.metadata (2.1 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached ultralytics-8.3.105-py3-none-any.whl.metadata (37 kB)
  Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached charset_normalizer-3.4.1-cp312-cp312-win_amd64.whl.metadata (36 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.1.31-py3-none-any.whl.metadata (2.5 kB)
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl.metadata (794 bytes)
  Using cached ultralytics_thop-2.0.14-py3-none-any.whl.metadata (9.4 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached PyYAML-6.0.2-cp312-cp312-win_amd

In [8]:
import os
import sys

yolo_path = "c:/Users/mague/Downloads/Object_detectorVC/yolov5"
os.chdir(yolo_path)

if yolo_path not in sys.path:
    sys.path.append(yolo_path)

In [12]:
import os
import sys
import torch
import cv2
import mediapipe as mp
import numpy as np
from pathlib import Path

# 📁 Ruta donde está tu carpeta clonada de YOLOv5
yolo_path = "c:/Users/mague/Downloads/Object_detectorVC/yolov5"
os.chdir(yolo_path)
if yolo_path not in sys.path:
    sys.path.append(yolo_path)

# ✅ Importaciones desde YOLOv5
from yolov5.models.common import DetectMultiBackend
from yolov5.utils.general import non_max_suppression, scale_boxes
from yolov5.utils.torch_utils import select_device
from yolov5.utils.augmentations import letterbox

# 🔧 Inicializar YOLOv5
device = select_device('')
model = DetectMultiBackend('./yolov5s.pt', device=device)
model.eval()

# 🕺 Inicializar MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False)
mp_drawing = mp.solutions.drawing_utils

# 📹 Cargar el video
input_path = "../assets/danceVideo.mp4"
cap = cv2.VideoCapture(input_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# 🎥 Inicializar salidas
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out_pose = cv2.VideoWriter("../assets/silhouette_dance_video.mp4", fourcc, fps, (width, height))
out_black = cv2.VideoWriter("../assets/silhouette_dance_black.mp4", fourcc, fps, (width, height))

# 🔁 Procesar video frame por frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    original = frame.copy()
    black_frame = np.zeros_like(frame)

    # 📏 Preparar frame para YOLOv5
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img_resized = letterbox(img, new_shape=640)[0]
    img_yolo = torch.from_numpy(img_resized).permute(2, 0, 1).float().to(device) / 255.0
    img_yolo = img_yolo.unsqueeze(0)

    # 🧠 Inferencia con YOLO
    pred = model(img_yolo)
    pred = non_max_suppression(pred, conf_thres=0.5, classes=[0])  # clase 0 = persona

    for det in pred:
        if det is not None and len(det):
            det[:, :4] = scale_boxes(img_resized.shape[:2], det[:, :4], frame.shape).round()

            for *xyxy, conf, cls in det:
                x1, y1, x2, y2 = map(int, xyxy)
                person_roi = frame[y1:y2, x1:x2]
                if person_roi.size == 0:
                    continue

                # 🧍 Estimar pose de la persona detectada
                person_rgb = cv2.cvtColor(person_roi, cv2.COLOR_BGR2RGB)
                result = pose.process(person_rgb)

                if result.pose_landmarks:
                    for landmark in result.pose_landmarks.landmark:
                        lx = int(x1 + landmark.x * (x2 - x1))
                        ly = int(y1 + landmark.y * (y2 - y1))
                        cv2.circle(original, (lx, ly), 4, (0, 255, 0), -1)
                        cv2.circle(black_frame, (lx, ly), 4, (255, 255, 255), -1)

    out_pose.write(original)
    out_black.write(black_frame)

# 🧹 Liberar recursos
cap.release()
out_pose.release()
out_black.release()
cv2.destroyAllWindows()

YOLOv5  v7.0-411-gf4d8a84c Python-3.12.5 torch-2.6.0+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
